In [27]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from scipy.stats import skew, kurtosis
    

In [28]:
base_dir = "./data"

def load_activity_data(directory, label):
    files = os.listdir(directory)
    data_frames = [pd.read_csv(os.path.join(directory, file)) for file in files]
    combined_data = pd.concat(data_frames, ignore_index=True)
    combined_data['label'] = label
    return combined_data

data_idle = load_activity_data(os.path.join(base_dir, 'idle'), 'idle')
data_running = load_activity_data(os.path.join(base_dir, 'running'), 'running')
data_stairs = load_activity_data(os.path.join(base_dir, 'stairs'), 'stairs')
data_walking = load_activity_data(os.path.join(base_dir, 'walking'), 'walking')

data = pd.concat([data_idle, data_running, data_stairs, data_walking], ignore_index=True)
    

In [29]:
def create_time_windows(df, window_size=30, step_size=15):
    n_rows = df.shape[0]
    windows = []
    for start in range(0, n_rows, step_size):
        end = start + window_size
        if end <= n_rows:
            windows.append(df.iloc[start:end])
    return windows

def compute_time_domain_features(window):
    features = {}
    features['mean_x'] = window['accelerometer_X'].mean()
    features['mean_y'] = window['accelerometer_Y'].mean()
    features['mean_z'] = window['accelerometer_Z'].mean()
    features['std_x'] = window['accelerometer_X'].std()
    features['std_y'] = window['accelerometer_Y'].std()
    features['std_z'] = window['accelerometer_Z'].std()
    features['skew_x'] = skew(window['accelerometer_X'])
    features['skew_y'] = skew(window['accelerometer_Y'])
    features['skew_z'] = skew(window['accelerometer_Z'])
    features['kurt_x'] = kurtosis(window['accelerometer_X'])
    features['kurt_y'] = kurtosis(window['accelerometer_Y'])
    features['kurt_z'] = kurtosis(window['accelerometer_Z'])
    return pd.Series(features)
    

In [30]:
features = []
labels = []
for label in data['label'].unique():
    activity_data = data[data['label'] == label]
    windows = create_time_windows(activity_data, window_size=30, step_size=15)
    for window in windows:
        features.append(compute_time_domain_features(window))
        labels.append(label)

features_df = pd.DataFrame(features)
features_df['label'] = labels
    

In [31]:
X = features_df.drop(columns=['label'])
y = features_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
    

In [32]:
report_svm = classification_report(y_test, y_pred_svm)
report_rf = classification_report(y_test, y_pred_rf)

print("SVM Classification Report:\n", report_svm)
print("Random Forest Classification Report:\n", report_rf)
    

SVM Classification Report:
               precision    recall  f1-score   support

        idle       1.00      1.00      1.00       415
     running       1.00      1.00      1.00      1363
      stairs       0.88      0.65      0.75        66
     walking       0.97      0.99      0.98       740

    accuracy                           0.99      2584
   macro avg       0.96      0.91      0.93      2584
weighted avg       0.99      0.99      0.99      2584

Random Forest Classification Report:
               precision    recall  f1-score   support

        idle       1.00      1.00      1.00       415
     running       1.00      1.00      1.00      1363
      stairs       0.98      0.89      0.94        66
     walking       0.99      1.00      0.99       740

    accuracy                           1.00      2584
   macro avg       0.99      0.97      0.98      2584
weighted avg       1.00      1.00      1.00      2584

